In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import training_data_cleaning_joining as clean

%matplotlib inline

In [2]:
all_train = clean.merge_all_train_data()

test_data = clean.load_test_data()
sample_submission = clean.load_submission_file()

# Using monthly aggregated sales

In [3]:
all_train.head().T

,0,1,2,3,4
date,2013-01-02 00:00:00,2013-01-02 00:00:00,2013-01-03 00:00:00,2013-01-20 00:00:00,2013-01-23 00:00:00
date_block_num,0,0,0,0,0
year,2013,2013,2013,2013,2013
month,1,1,1,1,1
day_of_month,2,2,3,20,23
month_year,2013-01,2013-01,2013-01,2013-01,2013-01
shop_id,59,25,25,25,25
shop_name,"Ярославль ТЦ ""Альтаир""","Москва ТРК ""Атриум""","Москва ТРК ""Атриум""","Москва ТРК ""Атриум""","Москва ТРК ""Атриум"""
shop_name_eng,"yaroslavl shopping center ""altair""","moscow trk ""atrium""","moscow trk ""atrium""","moscow trk ""atrium""","moscow trk ""atrium"""
shop_city,yaroslavl,moscow,moscow,moscow,moscow


In [4]:
sales_by_month = all_train.copy().groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()
sales_by_month.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)

In [5]:
sales_by_month['shop_id'].nunique()

60

In [6]:
sales_by_month['item_id'].nunique()

21807

## Predict using last month sales

In [7]:
sales_last_month = sales_by_month[sales_by_month['date_block_num']==33].reset_index().copy()
sales_last_month.drop(columns=['index', 'date_block_num'], inplace=True)

In [8]:
sales_last_month.head()

,shop_id,item_id,item_cnt_month
0,2,31,1
1,2,486,3
2,2,787,1
3,2,794,1
4,2,968,1


In [9]:
print('number of shops with sales in last month:', sales_last_month['shop_id'].nunique())
print('number of items sold in last month:', sales_last_month['item_id'].nunique())
print('number of shop-item-combinations: %d, out of %d possible' %(
    len(sales_last_month), sales_last_month['shop_id'].nunique()*sales_last_month['item_id'].nunique()))

number of shops with sales in last month: 44
number of items sold in last month: 5413
number of shop-item-combinations: 31531, out of 238172 possible


In [10]:
# make grid dataframe with only shop_ids and item_ids in the test data, with each item_id occuring in all shop_ids.
submit = pd.merge(test_data, sales_last_month, how='left', on=['shop_id', 'item_id'])

# set item_cnt to zero for items not sold in shop
submit.fillna(0, inplace=True)

In [11]:
submit.head()

,ID,shop_id,item_id,item_cnt_month
0,0,5,5037,0.0
1,1,5,5320,0.0
2,2,5,5233,1.0
3,3,5,5232,0.0
4,4,5,5268,0.0


In [12]:
print('predicted item_cnt_month > 20: %.2f%%'
      %np.round(100 * submit[submit['item_cnt_month']>20].shape[0]/len(submit), 2))

print('predicted item_cnt_month < 0: %.4f%%'
      %np.round(100 * submit[submit['item_cnt_month']<0].shape[0]/len(submit), 4))

predicted item_cnt_month > 20: 0.09%
predicted item_cnt_month < 0: 0.0037%


In [13]:
# truncate predicted item_cnt_month to between 0 and 20 to match test data
submit.loc[submit['item_cnt_month']>20, 'item_cnt_month'] = 20
submit.loc[submit['item_cnt_month']<0, 'item_cnt_month'] = 0

In [14]:
# sanity check
submit['item_cnt_month'].describe()

count    214200.000000
mean          0.255649
std           1.089856
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          20.000000
Name: item_cnt_month, dtype: float64

In [15]:
# submit[['ID','item_cnt_month']].to_csv('submission_last_month_sales.csv', index=False)